# Secure sorting networks explained

In this notebook, we develop some MPC protocols for securely sorting lists of secret-shared numbers. Concretely, we will show how to define functions sorting lists of secure MPyC integers into ascending order. The values represented by the secure integers and their relative order should remain completely secret.

The explanation below assumes some basic familiarity with the MPyC framework for secure computation. Our main goal is to show how existing Python code for (oblivious) sorting can be used to implement a secure MPC sorting protocol using the `mpyc` package. The modifications to the existing code are very limited.

## Sorting networks

[Sorting networks](https://en.wikipedia.org/wiki/Sorting_network) are a classical type of comparison-based sorting algorithms. The basic operation (or, gate) in a sorting network is the *compare&swap* operation, which puts any two list elements $x[i]$ and $x[j]$, $i<j$, in ascending order. That is, only if $x[i]>x[j]$, elements $x[i]$ and $x[j]$ are swapped, and otherwise the compare&swap operation leaves the list unchanged. 

A sorting network specifies the exact sequence of compare&swap operations to be applied to a list of a given length $n$. The particular sequence depends only on $n$, the length of the input list. Even when the input list is already in ascending order, the sorting network will perform exactly as many---and actually the same---compare&swap operations as when the input list would be in descending order. 

For example, to sort a list of three numbers, one needs to perform three compare&swap operations with indices $(i,j)$ equal to $(0,1)$, then $(1,2)$, and finally once more $(0,1)$.

Below, we will use odd-even merge sort and bitonic sort, which are two well-known practical sorting networks. 

## MPyC setup

A simple MPyC setup using 32-bit (default) secure MPyC integers suffices for the purpose of this demonstration.

At this point we also import the Python `traceback` module for later use.

In [2]:
from mpyc.runtime import mpc    # load MPyC
secint = mpc.SecInt()           # 32-bit secure MPyC integers
mpc.run(mpc.start())            # required only when run with multiple parties
import traceback                # to show some suppressed error messages
import math

2023-04-04 12:05:42,517 Start MPyC runtime v0.9


## Odd-even merge sort

Odd-even merge sort is an elegant, but somewhat intricate, sorting network. The details are nicely explained in the Wikipedia article [Batcher's Odd-Even Mergesort](https://en.wikipedia.org/wiki/Batcher_odd–even_mergesort). 

For our purposes, however, there is no need to understand exactly how this particular sorting network works. The only thing that we need to do is to grab the following  [example Python code](https://en.wikipedia.org/w/index.php?title=Batcher_odd%E2%80%93even_mergesort&oldid=969926478#Example_code) from this Wikipedia article.

In [8]:
def binary_search_insecure(x, a):
    low = 0
    high = len(x)-1
    if x[0] == a:
            return 0
    elif x[len(x)-1] == a:
        return len(x)-1
    else:
        while(low <= high):
            mid = int((low+high)/2)
            if x[mid]==a:
                return mid
            elif x[mid] > a:
                high = mid -1
            elif x[mid] < a:
                low = mid +1
        return -1  #returns -1 if the target does not exist
    
def binary_search(x, a):
    low = 0
    high = len(x)-1
   
    searching = True
    array_length = len(x)
    searches = int(math.log(array_length, 2))

    mid = -1

    for i in range(searches):
        mid = (low+high)//2
        high = mpc.if_else(x[mid] > a, mid - 1, high)
        low = mpc.if_else(x[mid] < a, mid + 1, low)

    mpc.if_else(x[mid] != a, -1, mid)

    return mid  #returns -1 if the target does not exist

We run the code on a simple example. Note that this code assumes that the length of the input list is an integral power of two.

In [9]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
a = 7
index = binary_search_insecure(x, a)
print(index)

6


In [10]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
a = 7
index = binary_search(x, a)
print(index)

6


We try to run this code on a list of secure MPyC integers.

In [11]:
x = list(map(secint, [2, 4, 3, 5, 6, 1, 7, 8]))
a = 7
try:
    index = binary_search_insecure(x, a)
    print(index)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_6515/4227598106.py", line 4, in <module>
    index = binary_search_insecure(x, a)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_6515/1103545969.py", line 4, in binary_search_insecure
    if x[0] == a:
       ^^^^^^^^^
  File "/home/jos/bin/mpyc/mpyc/sectypes.py", line 69, in __bool__
    raise TypeError('cannot use secure type in Boolean expressions')
TypeError: cannot use secure type in Boolean expressions


In [12]:
x = list(map(secint, [2, 4, 3, 5, 6, 1, 7, 8]))
x = mpc.seclist(x)
a = 7
try:
    index = binary_search(x, a)
    print(index)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_6515/1852210698.py", line 5, in <module>
    index = binary_search(x, a)
            ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_6515/1103545969.py", line 34, in binary_search
    if x[mid] != a:
       ^^^^^^^^^^^
  File "/home/jos/bin/mpyc/mpyc/sectypes.py", line 69, in __bool__
    raise TypeError('cannot use secure type in Boolean expressions')
TypeError: cannot use secure type in Boolean expressions


In [58]:
mpc.run(mpc.shutdown())   # required only when run with multiple parties

2023-03-31 17:04:42,798 Stop MPyC -- elapsed time: 1:38:15.644|bytes sent: 0


The Python script [sort.py](sort.py) demos MPyC's built-in sorting method, which uses Batcher's merge-exchange sort (nonrecursive version of odd-even merge sort).